# Consensus algorithm

In this example we'll show how to run a simple consensus algorithm.

## Background

We consider a network of `num_clients` peer clients. Each client $i$ maintains some local value $x_i^t$ and is connected to a subset of the other clients in the network, defined as the set of its neighbors $\mathcal{N}_i$.

The goal of all the clients is to reach consensus on their local values by communicating with their peers for a certain number of iterations $T$. 
In other terms, they want to cooperatively compute a weighted average of their initial values $x_i^0$.

The consensus algorithm works by having each client updating its local value as

$$x_i^{t+1} = \sum_{j\in\mathcal{N}_i}a_{ji}x_j^t$$

where $a_{ji}$ is the weight associated by client $i$ to client $j$ and $\sum_{j=1}^N a_{ji}=1$ for all $i$.

## Setup
Let's assume we have $N=5$ clients and generate a random configuration.

In [ ]:
from nvflare.app_opt.p2p.utils.config_generator import generate_random_network
from nvflare.app_opt.p2p.types import Config

# generate random config
num_clients=5
network, adjacency_matrix = generate_random_network(num_clients=num_clients)
config = Config(network=network, extra={"iterations": 50})

By inspecting the config we can see the initial values of each client as well as the network topology.

In [ ]:
from rich import print

print(config)

## Instantiating and running the distributed algorithm

Now that we have all the configurations, we can instantiate the clients and a controller.

In [ ]:
from nvflare.job_config.api import FedJob
from nvflare.app_opt.p2p.executors import ConsensusExecutor
from nvflare.app_opt.p2p.controllers import DistOptController

# Create job
job = FedJob(name="consensus")

# send controller to server
controller = DistOptController(config=config)
job.to_server(controller)

# Add clients
for i in range(num_clients):
    executor = ConsensusExecutor() # the executor will be initialized with a random value
    job.to(executor, f"site-{i+1}")

Finally, we can run the algorithm

In [ ]:
# run
job.export_job("./tmp/job_configs")
job.simulator_run("./tmp/runs/consensus")

Now, since in this toy example we have access to both the intial values and the adjacency matrix, we can compute the consensus value at which the clients will converge.

In [ ]:
import torch

from nvflare.app_opt.p2p.utils.topology import get_matrix_steady_state

history = {
    f"site-{i+1}": torch.load(f"tmp/runs/consensus/site-{i+1}/value_sequence.pt") for i in range(num_clients)
}
x0=torch.tensor([h[0] for name, h in history.items()]).t()
consensus_value = x0 @ get_matrix_steady_state(adjacency_matrix)
print(f"Consensus value: {consensus_value}")

By inspecting the history/results, we can easily see that all the clients successfully converged to the expected consensus value. 

In [ ]:
import matplotlib.pyplot as plt

# plot results
plt.figure()
for i in range(num_clients):
    plt.plot(history[f"site-{i+1}"], label=f"site-{i+1}")
plt.axhline(y=consensus_value, color='k', linestyle="--", label="Consensus value")
plt.legend()
plt.title("Evolution of local values")
plt.show()